# Spark

### Introduction:

This time you will create the data.



### Step 1. Import the necessary libraries

In [1]:
import numpy as np
import os
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col , when , lit , udf ,to_timestamp
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import findspark
findspark.init()

# import findspark
# findspark.add_packages('mysql:mysql-connector-java:8.0.26')



In [2]:
sc = SparkContext(appName="mysqltest")
sqlContext = SQLContext(sc)
spark = SparkSession.builder.master("local").appName("PySpark_MySQL_test").getOrCreate()


In [3]:
# tableDF = sqlContext.read.format("jdbc")\
# .option("driver","com.mysql.jdbc.Driver")\
# .option("url","jdbc:mysql://localhost:3306/hitachi_vantara_report")\
# .option("dbtable", "user")\
# .option("user", "root")\
# .option("password", "locked1641999")\
# .load()

df_user = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/hitachi_vantara_report") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "user") \
    .option("user", "root").option("password", "locked1641999").load()
df_PM = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/hitachi_vantara_report") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "project_management") \
    .option("user", "root").option("password", "locked1641999").load()
df_report = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/hitachi_vantara_report") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "report") \
    .option("user", "root").option("password", "locked1641999").load()
df_comment = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/hitachi_vantara_report") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "comment") \
    .option("user", "root").option("password", "locked1641999").load()
df_user.show()

+---+--------------------+--------------------+--------+--------+
| id|               email|            fullname|is_admin|password|
+---+--------------------+--------------------+--------+--------+
|  1|khang.nguyen3@hit...|Nguyễn Đăng Nam K...|       1|  123abc|
|  2|thuan.nguyen1@hit...|    Nguyễn Đức Thuận|       1|     123|
|  3|khuong.hoang@hita...|        Hoàng Khương|       1|     123|
|  4|hau.nguyen1@hitac...|     Nguyễn Phúc Hậu|       1|     123|
+---+--------------------+--------------------+--------+--------+



In [4]:
df_PM.show()

+---+---------------+-----------+---------------+--------------------+----------------+-------------------+
| id|  customer_name|description|project_lead_id|        project_name|    project_lead|          create_at|
+---+---------------+-----------+---------------+--------------------+----------------+-------------------+
|  1|Nguyễn Hồng Sơn|           |              3|               Vcaas|    Hoàng Khương|               null|
|  2|Nguyễn Hồng Sơn|           |              2|      Project Intern|Nguyễn Đức Thuận|               null|
|  3|Nguyễn Hồng Sơn|           |              4|Data Science Project| Nguyễn Phúc Hậu|2021/07/19 20:07:40|
+---+---------------+-----------+---------------+--------------------+----------------+-------------------+



In [5]:
df_report.show()

+---+----------+------------------+--------------------+-------+-------------------+
| id|project_id|report_template_id|         report_data|user_id|          create_at|
+---+----------+------------------+--------------------+-------+-------------------+
|  1|         1|                 1|{"project_lead": ...|      1|               null|
|  3|         3|                 1|{"project_lead": ...|      1|2021/07/19 20:09:33|
|  4|         2|                 2|{"abc": "sdsds", ...|      1|2021/07/21 11:11:24|
+---+----------+------------------+--------------------+-------+-------------------+



In [6]:
df_comment.show()

+---+--------+---------+-------+------------+
| id| content|report_id|user_id|   user_name|
+---+--------+---------+-------+------------+
|  1|Good Job|        1|      3|Hoàng Khương|
+---+--------+---------+-------+------------+



In [7]:
column = ['id' , 'email' , 'fullname' , 'is_admin' , 'password']
newUser = spark.createDataFrame([(4,'tai.pham@hitachivantara.com','Phạm Đình Tài',0,'123abc')] , column)
newUser.show()

+---+--------------------+-------------+--------+--------+
| id|               email|     fullname|is_admin|password|
+---+--------------------+-------------+--------+--------+
|  4|tai.pham@hitachiv...|Phạm Đình Tài|       0|  123abc|
+---+--------------------+-------------+--------+--------+



In [8]:
df_user = df_user.union(newUser)
df_user.show()

+---+--------------------+--------------------+--------+--------+
| id|               email|            fullname|is_admin|password|
+---+--------------------+--------------------+--------+--------+
|  1|khang.nguyen3@hit...|Nguyễn Đăng Nam K...|       1|  123abc|
|  2|thuan.nguyen1@hit...|    Nguyễn Đức Thuận|       1|     123|
|  3|khuong.hoang@hita...|        Hoàng Khương|       1|     123|
|  4|hau.nguyen1@hitac...|     Nguyễn Phúc Hậu|       1|     123|
|  4|tai.pham@hitachiv...|       Phạm Đình Tài|       0|  123abc|
+---+--------------------+--------------------+--------+--------+



In [9]:
df_user = df_user.withColumn('id' , when(df_user['is_admin'] == 0 , 5).otherwise(df_user['id']))
df_user.show()

+---+--------------------+--------------------+--------+--------+
| id|               email|            fullname|is_admin|password|
+---+--------------------+--------------------+--------+--------+
|  1|khang.nguyen3@hit...|Nguyễn Đăng Nam K...|       1|  123abc|
|  2|thuan.nguyen1@hit...|    Nguyễn Đức Thuận|       1|     123|
|  3|khuong.hoang@hita...|        Hoàng Khương|       1|     123|
|  4|hau.nguyen1@hitac...|     Nguyễn Phúc Hậu|       1|     123|
|  5|tai.pham@hitachiv...|       Phạm Đình Tài|       0|  123abc|
+---+--------------------+--------------------+--------+--------+



In [10]:
df_user = df_user.withColumnRenamed('id' , 'user_id')
df_user.show()

+-------+--------------------+--------------------+--------+--------+
|user_id|               email|            fullname|is_admin|password|
+-------+--------------------+--------------------+--------+--------+
|      1|khang.nguyen3@hit...|Nguyễn Đăng Nam K...|       1|  123abc|
|      2|thuan.nguyen1@hit...|    Nguyễn Đức Thuận|       1|     123|
|      3|khuong.hoang@hita...|        Hoàng Khương|       1|     123|
|      4|hau.nguyen1@hitac...|     Nguyễn Phúc Hậu|       1|     123|
|      5|tai.pham@hitachiv...|       Phạm Đình Tài|       0|  123abc|
+-------+--------------------+--------------------+--------+--------+



In [11]:
df_user_comment = df_user.join(df_comment , ['user_id'])
df_user_comment.show()

+-------+--------------------+------------+--------+--------+---+--------+---------+------------+
|user_id|               email|    fullname|is_admin|password| id| content|report_id|   user_name|
+-------+--------------------+------------+--------+--------+---+--------+---------+------------+
|      3|khuong.hoang@hita...|Hoàng Khương|       1|     123|  1|Good Job|        1|Hoàng Khương|
+-------+--------------------+------------+--------+--------+---+--------+---------+------------+



In [12]:
df_user_report  = df_user.join(df_report , ['user_id'])
df_user_report = df_user_report.drop('user_id' , 'password' , 'is_admin' , 'id')
df_user_report.show()

+--------------------+--------------------+----------+------------------+--------------------+-------------------+
|               email|            fullname|project_id|report_template_id|         report_data|          create_at|
+--------------------+--------------------+----------+------------------+--------------------+-------------------+
|khang.nguyen3@hit...|Nguyễn Đăng Nam K...|         1|                 1|{"project_lead": ...|               null|
|khang.nguyen3@hit...|Nguyễn Đăng Nam K...|         3|                 1|{"project_lead": ...|2021/07/19 20:09:33|
|khang.nguyen3@hit...|Nguyễn Đăng Nam K...|         2|                 2|{"abc": "sdsds", ...|2021/07/21 11:11:24|
+--------------------+--------------------+----------+------------------+--------------------+-------------------+



In [13]:
df_user_report.write.format('jdbc').options(
      url='jdbc:mysql://localhost : 3306/hitachi_vantara_report',
      driver='com.mysql.jdbc.Driver',
      dbtable='spark_import',
      user='root',
      password='locked1641999')\
      .mode('append')\
      .save()

In [13]:
df_error_data_1 = spark.read.option("header",True).csv('error_data_part1.csv')
df_error_data_2 = spark.read.option("header",True).csv('error_data_part2.csv')
df_product_data_1 = spark.read.option("header",True).csv('product_raw_data_part1.csv')
df_product_data_2 = spark.read.option("header",True).csv('product_raw_data_part2.csv')

In [14]:
df_error_data_1.show()

+----------+------+--------------+----+
|Unnamed: 0|errors|    product_id|code|
+----------+------+--------------+----+
|         0|    96| PRODUCT_14111|BLOK|
|         1|     8|   PRODUCT_120|BLOK|
|         2|    96|    PRODUCT_41|FQCH|
|         3|     2|    PRODUCT_14|FQCH|
|         4|     5|   PRODUCT_120|FQCH|
|         5|     8| PRODUCT_14222|BLOK|
|         6|     3| PRODUCT_13333|BLOK|
|         7|     5|    PRODUCT_37|BLOK|
|         8|     5|    PRODUCT_34|BLOK|
|         9|     6|PRODUCT_145555|EMAN|
|        10|     2|    PRODUCT_16|EMAN|
|        11|     7| PRODUCT_14111|FQCH|
|        12|     1|    PRODUCT_29|EMAN|
|        13|     1|    PRODUCT_33|BLOK|
|        14|     1|    PRODUCT_33|BPAK|
|        15|    31|    PRODUCT_63|BPAK|
|        16|    48|PRODUCT_149999|FQCH|
|        17|    48|PRODUCT_146666|EMAN|
|        18|     1|   PRODUCT_176|FQCH|
|        19|    13|   PRODUCT_145|BPAK|
+----------+------+--------------+----+
only showing top 20 rows



In [15]:
df_error_data_2.show()

+----------+------+-------------+----+
|Unnamed: 0|errors|   product_id|code|
+----------+------+-------------+----+
|         0|     1|PRODUCT_10041|FQCH|
|         1|     1|PRODUCT_10033|BPAK|
|         2|     2|PRODUCT_10081|BPAK|
|         3|     1|PRODUCT_10109|BPAK|
|         4|    17|PRODUCT_20041|FQCH|
|         5|     1|PRODUCT_20042|BLOK|
|         6|     2|PRODUCT_10099|BLOK|
|         7|     4|PRODUCT_10124|EMAN|
|         8|    60|PRODUCT_20043|FQCH|
|         9|     3|PRODUCT_20044|BPAK|
|        10|     1|PRODUCT_10094|EMAN|
|        11|     2|PRODUCT_10053|FQCH|
|        12|     2|PRODUCT_10015|BLOK|
|        13|     1|PRODUCT_20045|EMAN|
|        14|     3|PRODUCT_10149|FQCH|
|        15|     7|PRODUCT_20046|BLOK|
|        16|     1|PRODUCT_10137|FQCH|
|        17|     1|PRODUCT_10141|FQCH|
|        18|     1|PRODUCT_20047|BLOK|
|        19|     1|PRODUCT_10122|BPAK|
+----------+------+-------------+----+
only showing top 20 rows



In [16]:
df_error_data = df_error_data_1.union(df_error_data_2)
df_error_data = df_error_data.drop('Unnamed: 0')
df_error_data.show()

+------+--------------+----+
|errors|    product_id|code|
+------+--------------+----+
|    96| PRODUCT_14111|BLOK|
|     8|   PRODUCT_120|BLOK|
|    96|    PRODUCT_41|FQCH|
|     2|    PRODUCT_14|FQCH|
|     5|   PRODUCT_120|FQCH|
|     8| PRODUCT_14222|BLOK|
|     3| PRODUCT_13333|BLOK|
|     5|    PRODUCT_37|BLOK|
|     5|    PRODUCT_34|BLOK|
|     6|PRODUCT_145555|EMAN|
|     2|    PRODUCT_16|EMAN|
|     7| PRODUCT_14111|FQCH|
|     1|    PRODUCT_29|EMAN|
|     1|    PRODUCT_33|BLOK|
|     1|    PRODUCT_33|BPAK|
|    31|    PRODUCT_63|BPAK|
|    48|PRODUCT_149999|FQCH|
|    48|PRODUCT_146666|EMAN|
|     1|   PRODUCT_176|FQCH|
|    13|   PRODUCT_145|BPAK|
+------+--------------+----+
only showing top 20 rows



In [17]:
df_product_data_1.show()

+---+------------+-------------------+-------------------+-------------+
|_c0|machine_name|            in_time|           out_time|   product_id|
+---+------------+-------------------+-------------------+-------------+
|  0| MACHINE_002|2014-12-02 00-42-46|2014-12-02 01-47-57|PRODUCT_10000|
|  1| MACHINE_002|2014-12-01 23-40-12|2014-12-02 00-42-36|PRODUCT_10001|
|  2| MACHINE_002|2014-12-02 09-39-49|2014-12-02 12-56-12|PRODUCT_10002|
|  3| MACHINE_002|2014-12-02 04-40-50|2014-12-02 05-39-32|PRODUCT_10003|
|  4| MACHINE_002|2014-12-02 15-25-44|2014-12-02 16-58-26|PRODUCT_10004|
|  5| MACHINE_002|2014-12-02 03-23-05|2014-12-02 04-40-40|PRODUCT_10005|
|  6| MACHINE_002|2014-12-02 01-48-07|2014-12-02 03-22-55|PRODUCT_10006|
|  7| MACHINE_002|2014-12-02 05-39-41|2014-12-02 06-58-17|PRODUCT_10007|
|  8| MACHINE_002|2014-12-02 14-08-17|2014-12-02 15-25-18|PRODUCT_10008|
|  9| MACHINE_002|2014-12-02 12-56-21|2014-12-02 14-06-47|PRODUCT_10009|
| 10| MACHINE_002|2014-12-01 21-28-45|2014-12-01 22

In [18]:
df_product_data_2.show()

+---+------------+-------------------+-------------------+----------+
|_c0|machine_name|           out_time|            in_time|product_id|
+---+------------+-------------------+-------------------+----------+
|  0| MACHINE_001|2014-12-01 23-54-18|2014-12-01 22-54-34| PRODUCT_1|
|  1| MACHINE_001|2014-12-02 09-41-20|2014-12-02 08-01-13| PRODUCT_2|
|  2| MACHINE_001|2014-12-02 11-43-00|2014-12-02 09-42-25| PRODUCT_3|
|  3| MACHINE_001|2014-12-02 04-37-19|2014-12-02 03-32-16| PRODUCT_4|
|  4| MACHINE_001|2014-12-02 14-14-46|2014-12-02 12-45-33| PRODUCT_5|
|  5| MACHINE_001|2014-12-02 02-10-19|2014-12-01 23-54-55| PRODUCT_6|
|  6| MACHINE_001|2014-12-02 15-39-18|2014-12-02 14-17-52| PRODUCT_7|
|  7| MACHINE_001|2014-12-02 16-41-45|2014-12-02 15-39-30| PRODUCT_8|
|  8| MACHINE_001|2014-12-02 08-00-52|2014-12-02 07-00-01| PRODUCT_9|
|  9| MACHINE_001|2014-12-02 05-41-26|2014-12-02 04-37-33|PRODUCT_10|
| 10| MACHINE_001|2014-12-02 06-15-40|2014-12-02 05-41-45|PRODUCT_11|
| 11| MACHINE_001|20

In [19]:
df_product_data = df_product_data_1.union(df_product_data_2)
df_product_data = df_product_data.drop('_c0')
df_product_data.show()

+------------+-------------------+-------------------+-------------+
|machine_name|            in_time|           out_time|   product_id|
+------------+-------------------+-------------------+-------------+
| MACHINE_002|2014-12-02 00-42-46|2014-12-02 01-47-57|PRODUCT_10000|
| MACHINE_002|2014-12-01 23-40-12|2014-12-02 00-42-36|PRODUCT_10001|
| MACHINE_002|2014-12-02 09-39-49|2014-12-02 12-56-12|PRODUCT_10002|
| MACHINE_002|2014-12-02 04-40-50|2014-12-02 05-39-32|PRODUCT_10003|
| MACHINE_002|2014-12-02 15-25-44|2014-12-02 16-58-26|PRODUCT_10004|
| MACHINE_002|2014-12-02 03-23-05|2014-12-02 04-40-40|PRODUCT_10005|
| MACHINE_002|2014-12-02 01-48-07|2014-12-02 03-22-55|PRODUCT_10006|
| MACHINE_002|2014-12-02 05-39-41|2014-12-02 06-58-17|PRODUCT_10007|
| MACHINE_002|2014-12-02 14-08-17|2014-12-02 15-25-18|PRODUCT_10008|
| MACHINE_002|2014-12-02 12-56-21|2014-12-02 14-06-47|PRODUCT_10009|
| MACHINE_002|2014-12-01 21-28-45|2014-12-01 22-02-35|PRODUCT_10010|
| MACHINE_002|2014-12-01 22-02-43|

In [20]:
df_result = df_product_data.join(df_error_data , ['product_id'])
df_result.show()

+-------------+------------+-------------------+-------------------+------+----+
|   product_id|machine_name|            in_time|           out_time|errors|code|
+-------------+------------+-------------------+-------------------+------+----+
|PRODUCT_10015| MACHINE_002|2014-12-03 14-01-56|2014-12-03 15-34-09|     2|BLOK|
|PRODUCT_10021| MACHINE_002|2014-12-03 07-59-24|2014-12-03 08-31-51|    23|FQCH|
|PRODUCT_10021| MACHINE_002|2014-12-03 07-59-24|2014-12-03 08-31-51|     4|BPAK|
|PRODUCT_10033| MACHINE_002|2014-12-03 11-53-30|2014-12-03 13-59-49|     1|BPAK|
|PRODUCT_10041| MACHINE_002|2014-12-03 00-13-52|2014-12-03 01-10-40|     1|FQCH|
|PRODUCT_10053| MACHINE_002|2014-12-03 08-29-06|2014-12-03 09-10-04|     2|FQCH|
|PRODUCT_10081| MACHINE_002|2014-12-05 20-48-56|2014-12-05 22-24-12|     2|BPAK|
|PRODUCT_10094| MACHINE_002|2014-12-06 23-49-18|2014-12-07 00-44-22|     1|EMAN|
|PRODUCT_10099| MACHINE_002|2014-12-06 18-55-23|2014-12-06 20-04-55|     2|BLOK|
|PRODUCT_10105| MACHINE_002|

In [21]:
df_result = df_result.withColumn('isError' , lit(0))
df_result.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- machine_name: string (nullable = true)
 |-- in_time: string (nullable = true)
 |-- out_time: string (nullable = true)
 |-- errors: string (nullable = true)
 |-- code: string (nullable = true)
 |-- isError: integer (nullable = false)



In [22]:
df_result = df_result.withColumn('errors' , df_result['errors'].cast('float'))
df_result.show()

+-------------+------------+-------------------+-------------------+------+----+-------+
|   product_id|machine_name|            in_time|           out_time|errors|code|isError|
+-------------+------------+-------------------+-------------------+------+----+-------+
|PRODUCT_10015| MACHINE_002|2014-12-03 14-01-56|2014-12-03 15-34-09|   2.0|BLOK|      0|
|PRODUCT_10021| MACHINE_002|2014-12-03 07-59-24|2014-12-03 08-31-51|  23.0|FQCH|      0|
|PRODUCT_10021| MACHINE_002|2014-12-03 07-59-24|2014-12-03 08-31-51|   4.0|BPAK|      0|
|PRODUCT_10033| MACHINE_002|2014-12-03 11-53-30|2014-12-03 13-59-49|   1.0|BPAK|      0|
|PRODUCT_10041| MACHINE_002|2014-12-03 00-13-52|2014-12-03 01-10-40|   1.0|FQCH|      0|
|PRODUCT_10053| MACHINE_002|2014-12-03 08-29-06|2014-12-03 09-10-04|   2.0|FQCH|      0|
|PRODUCT_10081| MACHINE_002|2014-12-05 20-48-56|2014-12-05 22-24-12|   2.0|BPAK|      0|
|PRODUCT_10094| MACHINE_002|2014-12-06 23-49-18|2014-12-07 00-44-22|   1.0|EMAN|      0|
|PRODUCT_10099| MACHI

In [51]:
df_result = df_result.withColumn('isError' , when(df_result['errors'] >= 96 , 1).otherwise(df_result['isError']))
df_result.show()

+-------------+------------+-------------------+-------------------+------+----+-------+
|   product_id|machine_name|            in_time|           out_time|errors|code|isError|
+-------------+------------+-------------------+-------------------+------+----+-------+
|PRODUCT_10015| MACHINE_002|2014-12-03 14-01-56|2014-12-03 15-34-09|   2.0|BLOK|      0|
|PRODUCT_10021| MACHINE_002|2014-12-03 07-59-24|2014-12-03 08-31-51|  23.0|FQCH|      0|
|PRODUCT_10021| MACHINE_002|2014-12-03 07-59-24|2014-12-03 08-31-51|   4.0|BPAK|      0|
|PRODUCT_10033| MACHINE_002|2014-12-03 11-53-30|2014-12-03 13-59-49|   1.0|BPAK|      0|
|PRODUCT_10041| MACHINE_002|2014-12-03 00-13-52|2014-12-03 01-10-40|   1.0|FQCH|      0|
|PRODUCT_10053| MACHINE_002|2014-12-03 08-29-06|2014-12-03 09-10-04|   2.0|FQCH|      0|
|PRODUCT_10081| MACHINE_002|2014-12-05 20-48-56|2014-12-05 22-24-12|   2.0|BPAK|      0|
|PRODUCT_10094| MACHINE_002|2014-12-06 23-49-18|2014-12-07 00-44-22|   1.0|EMAN|      0|
|PRODUCT_10099| MACHI

In [24]:
df_product_isError = df_result.filter(df_result['isError'] == 1)
df_product_isError.show()

+-------------+------------+-------------------+-------------------+------+----+-------+
|   product_id|machine_name|            in_time|           out_time|errors|code|isError|
+-------------+------------+-------------------+-------------------+------+----+-------+
|PRODUCT_10971| MACHINE_002|2015-01-11 18-57-58|2015-01-11 20-40-02|  99.0|BLOK|      1|
|PRODUCT_11413| MACHINE_002|2015-01-29 05-12-30|2015-01-29 06-20-01| 111.0|EMAN|      1|
|PRODUCT_11413| MACHINE_002|2015-01-29 05-12-30|2015-01-29 06-20-01| 111.0|FQCH|      1|
|PRODUCT_11622| MACHINE_002|2015-02-07 10-08-30|2015-02-07 11-32-26|  96.0|FQCH|      1|
|PRODUCT_14111| MACHINE_002|2015-06-03 13-11-59|2015-06-03 15-09-55|  96.0|BLOK|      1|
|   PRODUCT_41| MACHINE_001|2014-12-04 07-19-53|2014-12-04 07-00-27|  96.0|FQCH|      1|
|  PRODUCT_151| MACHINE_001|2014-12-10 16-30-12|2014-12-10 14-40-42| 104.0|EMAN|      1|
|  PRODUCT_439| MACHINE_001|2014-12-23 06-27-04|2014-12-23 05-14-20|  96.0|EMAN|      1|
|  PRODUCT_444| MACHI

In [27]:
df_product_isError.write.format('jdbc').options(
      url='jdbc:mysql://localhost : 3306/hitachi_vantara_report',
      driver='com.mysql.jdbc.Driver',
      dbtable='spark_import',
      user='root',
      password='locked1641999')\
      .mode('append')\
      .save()

In [25]:
arrCode = df_product_isError.select('code').collect()
def countErrorCode(*args) :
    result = {}
    arrArgument  = list(args)
    for arg in arrArgument:
        count = len([i.code for i in arrCode if i.code == arg])
        result[arg] = count
    return result

print(countErrorCode('BLOK' , 'EMAN' , 'FQCH'))

{'BLOK': 13, 'EMAN': 16, 'FQCH': 12}


In [26]:
arrErrorCode = df_product_isError.select('code' ,'errors').collect()
def sumErrorCode(*args) :
    result = {}
    arrArgument  = list(args)
    for arg in arrArgument:
        sumError = sum([i.errors for i in arrErrorCode if i.code == arg])
        result[arg] = sumError
    return result

print(sumErrorCode('BLOK' , 'EMAN' , 'FQCH'))

{'BLOK': 1251.0, 'EMAN': 1562.0, 'FQCH': 1167.0}


In [27]:
udfMergeColumn  = udf(lambda x , y :  '{} : {}'.format(x , y) , StringType())
df_product_isError.withColumn('merge_column' , udfMergeColumn('errors' , 'code')).show()
    

+-------------+------------+-------------------+-------------------+------+----+-------+------------+
|   product_id|machine_name|            in_time|           out_time|errors|code|isError|merge_column|
+-------------+------------+-------------------+-------------------+------+----+-------+------------+
|PRODUCT_10971| MACHINE_002|2015-01-11 18-57-58|2015-01-11 20-40-02|  99.0|BLOK|      1| 99.0 : BLOK|
|PRODUCT_11413| MACHINE_002|2015-01-29 05-12-30|2015-01-29 06-20-01| 111.0|EMAN|      1|111.0 : EMAN|
|PRODUCT_11413| MACHINE_002|2015-01-29 05-12-30|2015-01-29 06-20-01| 111.0|FQCH|      1|111.0 : FQCH|
|PRODUCT_11622| MACHINE_002|2015-02-07 10-08-30|2015-02-07 11-32-26|  96.0|FQCH|      1| 96.0 : FQCH|
|PRODUCT_14111| MACHINE_002|2015-06-03 13-11-59|2015-06-03 15-09-55|  96.0|BLOK|      1| 96.0 : BLOK|
|   PRODUCT_41| MACHINE_001|2014-12-04 07-19-53|2014-12-04 07-00-27|  96.0|FQCH|      1| 96.0 : FQCH|
|  PRODUCT_151| MACHINE_001|2014-12-10 16-30-12|2014-12-10 14-40-42| 104.0|EMAN|  

In [28]:
def formatTime(x) :
    arrayTime = x.split(' ')
    arrayTime[1] = arrayTime[1].replace('-' , ':')
    strTime = " ".join(arrayTime)
    return strTime
    
udfFormatTime = udf(formatTime , StringType())
df_product_isError = df_product_isError.withColumn('in_time' , udfFormatTime('in_time')) \
                                       .withColumn('out_time' , udfFormatTime('out_time'))
df_product_isError.show()

+-------------+------------+-------------------+-------------------+------+----+-------+
|   product_id|machine_name|            in_time|           out_time|errors|code|isError|
+-------------+------------+-------------------+-------------------+------+----+-------+
|PRODUCT_10971| MACHINE_002|2015-01-11 18:57:58|2015-01-11 20:40:02|  99.0|BLOK|      1|
|PRODUCT_11413| MACHINE_002|2015-01-29 05:12:30|2015-01-29 06:20:01| 111.0|EMAN|      1|
|PRODUCT_11413| MACHINE_002|2015-01-29 05:12:30|2015-01-29 06:20:01| 111.0|FQCH|      1|
|PRODUCT_11622| MACHINE_002|2015-02-07 10:08:30|2015-02-07 11:32:26|  96.0|FQCH|      1|
|PRODUCT_14111| MACHINE_002|2015-06-03 13:11:59|2015-06-03 15:09:55|  96.0|BLOK|      1|
|   PRODUCT_41| MACHINE_001|2014-12-04 07:19:53|2014-12-04 07:00:27|  96.0|FQCH|      1|
|  PRODUCT_151| MACHINE_001|2014-12-10 16:30:12|2014-12-10 14:40:42| 104.0|EMAN|      1|
|  PRODUCT_439| MACHINE_001|2014-12-23 06:27:04|2014-12-23 05:14:20|  96.0|EMAN|      1|
|  PRODUCT_444| MACHI

In [29]:
#dung format thi moi cast dc 
df_product_isError =df_product_isError.select('product_id','machine_name', 'errors' ,'code' , 'isError' , \
                    to_timestamp(df_product_isError['in_time'],'yyyy-MM-dd HH:mm:ss').alias('start_at'), \
                    to_timestamp(df_product_isError['out_time'],'yyyy-MM-dd HH:mm:ss').alias('end_at'))
df_product_isError.show()

+-------------+------------+------+----+-------+-------------------+-------------------+
|   product_id|machine_name|errors|code|isError|           start_at|             end_at|
+-------------+------------+------+----+-------+-------------------+-------------------+
|PRODUCT_10971| MACHINE_002|  99.0|BLOK|      1|2015-01-11 18:57:58|2015-01-11 20:40:02|
|PRODUCT_11413| MACHINE_002| 111.0|EMAN|      1|2015-01-29 05:12:30|2015-01-29 06:20:01|
|PRODUCT_11413| MACHINE_002| 111.0|FQCH|      1|2015-01-29 05:12:30|2015-01-29 06:20:01|
|PRODUCT_11622| MACHINE_002|  96.0|FQCH|      1|2015-02-07 10:08:30|2015-02-07 11:32:26|
|PRODUCT_14111| MACHINE_002|  96.0|BLOK|      1|2015-06-03 13:11:59|2015-06-03 15:09:55|
|   PRODUCT_41| MACHINE_001|  96.0|FQCH|      1|2014-12-04 07:19:53|2014-12-04 07:00:27|
|  PRODUCT_151| MACHINE_001| 104.0|EMAN|      1|2014-12-10 16:30:12|2014-12-10 14:40:42|
|  PRODUCT_439| MACHINE_001|  96.0|EMAN|      1|2014-12-23 06:27:04|2014-12-23 05:14:20|
|  PRODUCT_444| MACHI

In [30]:
df_product_isError.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- machine_name: string (nullable = true)
 |-- errors: float (nullable = true)
 |-- code: string (nullable = true)
 |-- isError: integer (nullable = false)
 |-- start_at: timestamp (nullable = true)
 |-- end_at: timestamp (nullable = true)



In [32]:
def timeLogWork(timeStart , timeEnd):
    time = timeEnd - timeStart
    hours  = round(time.seconds /3600)
    return '{} hours'.format(hours)
udfTimeLogWork = udf(timeLogWork, StringType())
df_product_isError = df_product_isError.withColumn('log_work' ,udfTimeLogWork('start_at' ,'end_at'))
df_product_isError.show()

+-------------+------------+------+----+-------+-------------------+-------------------+--------+
|   product_id|machine_name|errors|code|isError|           start_at|             end_at|log_work|
+-------------+------------+------+----+-------+-------------------+-------------------+--------+
|PRODUCT_10971| MACHINE_002|  99.0|BLOK|      1|2015-01-11 18:57:58|2015-01-11 20:40:02| 2 hours|
|PRODUCT_11413| MACHINE_002| 111.0|EMAN|      1|2015-01-29 05:12:30|2015-01-29 06:20:01| 1 hours|
|PRODUCT_11413| MACHINE_002| 111.0|FQCH|      1|2015-01-29 05:12:30|2015-01-29 06:20:01| 1 hours|
|PRODUCT_11622| MACHINE_002|  96.0|FQCH|      1|2015-02-07 10:08:30|2015-02-07 11:32:26| 1 hours|
|PRODUCT_14111| MACHINE_002|  96.0|BLOK|      1|2015-06-03 13:11:59|2015-06-03 15:09:55| 2 hours|
|   PRODUCT_41| MACHINE_001|  96.0|FQCH|      1|2014-12-04 07:19:53|2014-12-04 07:00:27|24 hours|
|  PRODUCT_151| MACHINE_001| 104.0|EMAN|      1|2014-12-10 16:30:12|2014-12-10 14:40:42|22 hours|
|  PRODUCT_439| MACH

In [46]:
rdd_product_isError = df_product_isError.rdd

In [47]:
rdd_product_isError.collect()

[Row(product_id='PRODUCT_10971', machine_name='MACHINE_002', errors=99.0, code='BLOK', isError=1, start_at=datetime.datetime(2015, 1, 11, 18, 57, 58), end_at=datetime.datetime(2015, 1, 11, 20, 40, 2), log_work='2 hours'),
 Row(product_id='PRODUCT_11413', machine_name='MACHINE_002', errors=111.0, code='EMAN', isError=1, start_at=datetime.datetime(2015, 1, 29, 5, 12, 30), end_at=datetime.datetime(2015, 1, 29, 6, 20, 1), log_work='1 hours'),
 Row(product_id='PRODUCT_11413', machine_name='MACHINE_002', errors=111.0, code='FQCH', isError=1, start_at=datetime.datetime(2015, 1, 29, 5, 12, 30), end_at=datetime.datetime(2015, 1, 29, 6, 20, 1), log_work='1 hours'),
 Row(product_id='PRODUCT_11622', machine_name='MACHINE_002', errors=96.0, code='FQCH', isError=1, start_at=datetime.datetime(2015, 2, 7, 10, 8, 30), end_at=datetime.datetime(2015, 2, 7, 11, 32, 26), log_work='1 hours'),
 Row(product_id='PRODUCT_14111', machine_name='MACHINE_002', errors=96.0, code='BLOK', isError=1, start_at=datetime.

In [45]:
rdd_product_isError.map(lambda x : (x.product_id , x.machine_name,x.errors,x.code,x.start_at ,x.log_work)) \
                    .collect()

[('PRODUCT_10971',
  'MACHINE_002',
  99.0,
  'BLOK',
  datetime.datetime(2015, 1, 11, 18, 57, 58),
  '2 hours'),
 ('PRODUCT_11413',
  'MACHINE_002',
  111.0,
  'EMAN',
  datetime.datetime(2015, 1, 29, 5, 12, 30),
  '1 hours'),
 ('PRODUCT_11413',
  'MACHINE_002',
  111.0,
  'FQCH',
  datetime.datetime(2015, 1, 29, 5, 12, 30),
  '1 hours'),
 ('PRODUCT_11622',
  'MACHINE_002',
  96.0,
  'FQCH',
  datetime.datetime(2015, 2, 7, 10, 8, 30),
  '1 hours'),
 ('PRODUCT_14111',
  'MACHINE_002',
  96.0,
  'BLOK',
  datetime.datetime(2015, 6, 3, 13, 11, 59),
  '2 hours'),
 ('PRODUCT_41',
  'MACHINE_001',
  96.0,
  'FQCH',
  datetime.datetime(2014, 12, 4, 7, 19, 53),
  '24 hours'),
 ('PRODUCT_151',
  'MACHINE_001',
  104.0,
  'EMAN',
  datetime.datetime(2014, 12, 10, 16, 30, 12),
  '22 hours'),
 ('PRODUCT_439',
  'MACHINE_001',
  96.0,
  'EMAN',
  datetime.datetime(2014, 12, 23, 6, 27, 4),
  '23 hours'),
 ('PRODUCT_444',
  'MACHINE_001',
  96.0,
  'EMAN',
  datetime.datetime(2014, 12, 23, 14, 14, 3